In [2]:
!pip install requests beautifulsoup4


[notice] A new release of pip is available: 23.2.1 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [18]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import math

def fetch_div_relative(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
        'Accept-Language': 'en-US,en;q=0.5',
        'Accept-Encoding': 'gzip, deflate',
        'Connection': 'keep-alive',
        'Upgrade-Insecure-Requests': '1'
    }
    
    try:
        response = requests.get(url, headers=headers, timeout=1.5)
        response.raise_for_status()  # Raises an HTTPError for bad responses
    except requests.exceptions.Timeout:
        print(f"Request timed out for URL: {url}")
        return None
    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
        return None

    soup = BeautifulSoup(response.text, 'html.parser')
    div_relative_content = soup.find('div', class_='relative')
    return div_relative_content

def extract_tables(div_relative_content):
    tables = div_relative_content.find_all('table')
    return tables

In [8]:
# URL of the webpage to scrape
url = "http://www.tabletennisleague.com/AGTTA/SessionGroupReportArchive/SessionGroupReport2024July23.HTM"
div_relative_content = fetch_div_relative(url)
# print("Div relative content:", div_relative_content)

if div_relative_content:
    tables = extract_tables(div_relative_content)
    print(type(tables[0]))
    print("Table 1:", tables[0])
    # for table in tables:
    #     print("Table:", table)


<class 'bs4.element.Tag'>
Table 1: <table cols="21" style="margin-right:7%">
<tr style="margin-bottom:-16px">
<td class="datacolumn1" style="color:red"><b>Hari Teja Narayanabhatla</b></td>
<td class="gameswlcolumn">9</td>
<td class="gameswlcolumn">5</td>
<td class="beforegw"></td>
<td class="gamesemcolumn1"></td>
<td class="gamesemcolumn1"></td>
<td class="gamesemcolumn1"></td>
<td class="gamesemcolumn">_</td>
<td class="gamesemcolumn"><a href="DisplayMatch.ASP?matchid=59675&amp;sessionnumber=5" title="Click to view details of this match">3</a></td>
<td class="gamesemcolumn"><a href="DisplayMatch.ASP?matchid=59674&amp;sessionnumber=5" title="Click to view details of this match">3</a></td>
<td class="gamesemcolumn"><a href="DisplayMatch.ASP?matchid=59672&amp;sessionnumber=5" title="Click to view details of this match">3</a></td>
<td class="aftergw"></td>
<td class="racolumn1"></td>
<td class="racolumn1"></td>
<td class="racolumn1"></td>
<td align="right" width="4%">_</td>
<td align="rig

In [5]:


# Extract data
data = []
rows = tables[0].find_all('tr')
for row in rows:
    cols = row.find_all('td')
    if cols:
        name = cols[0].get_text(strip=True)
        scores = [col.get_text(strip=True) for col in cols[7:11]]
        ratings = [col.get_text(strip=True) for col in cols[15:19]]
        opponents = [rows[i].find('td').get_text(strip=True) for i in range(len(rows)) if i != rows.index(row)]
        for opponent, score, rating in zip(opponents, scores, ratings):
            if score != "_" and rating != "_":
                data.append([name, opponent, score, rating])

# Create a DataFrame and save it to a CSV file
df = pd.DataFrame(data, columns=['Winner', 'Loser', 'Score', 'Rating Change'])
df


,Winner,Loser,Score,Rating Change
0,Hari Teja Narayanabhatla,Geetha Krishna,3,8
1,Hari Teja Narayanabhatla,Sunil Bandapally,3,4
2,Medha Krishna,Hari Teja Narayanabhatla,2,-8
3,Medha Krishna,Sunil Bandapally,3,4
4,Geetha Krishna,Hari Teja Narayanabhatla,2,-4
5,Geetha Krishna,Medha Krishna,0,-4
6,Sunil Bandapally,Hari Teja Narayanabhatla,1,-3
7,Sunil Bandapally,Medha Krishna,0,-3
8,Sunil Bandapally,Geetha Krishna,0,-7


#### This is the inner loop test to get the score and the rating

In [26]:
rows = tables[0].find_all('tr')
cols = rows[0].find_all('td')

scores_indices = [7, 8, 9, 10]
# Extract how many games the current player won
scores = [int(cols[i].get_text(strip=True)) if cols[i].get_text(strip=True).isdigit() else math.nan for i in scores_indices]

ratings_indices = [15, 16, 17, 18]
# Extract the ratings change of the current player
ratings = [int(cols[i].get_text(strip=True)) if cols[i].get_text(strip=True).isdigit() else math.nan for i in ratings_indices]

ratings


[nan, 8, 4, 3]

In [32]:
# Extract data
players_data = []
scores_data = []
ratings_data = []

for row in rows:
    cols = row.find_all('td')

    players_data.append(cols[0].get_text(strip=True))

    scores_indices = [7, 8, 9, 10]
    # Extract how many games the current player won
    scores = [int(cols[i].get_text(strip=True)) if cols[i].get_text(strip=True).lstrip("-").isdigit() else math.nan for i in scores_indices]

    ratings_indices = [15, 16, 17, 18]
    # Extract the ratings change of the current player
    ratings = [int(cols[i].get_text(strip=True)) if cols[i].get_text(strip=True).lstrip("-").isdigit() else math.nan for i in ratings_indices]

    scores_data.append(scores)
    ratings_data.append(ratings)

print("players_data: ", players_data)
print("scores_data: ", scores_data)
print("ratings_data: ", ratings_data)

"""
Prompt: 
I have 3 lists:
players_data:  ['Hari Teja Narayanabhatla', 'Medha Krishna', 'Geetha Krishna', 'Sunil Bandapally']
scores_data:  [[nan, 3, 3, 3], [2, nan, 3, 3], [2, 0, nan, 3], [1, 0, 0, nan]]
ratings_data:  [[nan, 8, 4, 3], [-8, nan, 4, 3], [-4, -4, nan, 7], [-3, -3, -7, nan]]

We can interpret these lists by the following: each of the inner list represents a player. For instance, scores_data[0] is from players_data[0], and she scored 3 against players_data[1], players_data[2], and players_data[3]. The same is with ratings_data, where they gained 
I would like to create a dataframe to store all of these 
"""

players_data:  ['Hari Teja Narayanabhatla', 'Medha Krishna', 'Geetha Krishna', 'Sunil Bandapally']
scores_data:  [[nan, 3, 3, 3], [2, nan, 3, 3], [2, 0, nan, 3], [1, 0, 0, nan]]
ratings_data:  [[nan, 8, 4, 3], [-8, nan, 4, 3], [-4, -4, nan, 7], [-3, -3, -7, nan]]
